# (Optional) Bias Field Correction + Pixel normalization

Iterates over the whole dataset and corrects the bias error of MRI images.
Bias Field correction might take a few hours

Everything is saved on new corrected dataset: mha_corrected_images

In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interactive, widgets, Layout, HBox, VBox, Label, interact, fixed
from IPython.display import display, clear_output
import itk
import SimpleITK as sitk
from concurrent.futures import ProcessPoolExecutor

In [4]:
def correct_bias_field(raw_img_sitk):
    # create head mask
    transformed = sitk.RescaleIntensity(raw_img_sitk, 0, 255)

    transformed = sitk.LiThreshold(transformed,0,1)

    head_mask = transformed

    #bias correction
    shrinkFactor = 4
    inputImage = raw_img_sitk

    inputImage = sitk.Shrink( raw_img_sitk, [ shrinkFactor ] * inputImage.GetDimension() )
    maskImage = sitk.Shrink( head_mask, [ shrinkFactor ] * inputImage.GetDimension() )

    bias_corrector = sitk.N4BiasFieldCorrectionImageFilter()

    corrected = bias_corrector.Execute(inputImage, maskImage)

    # get image corrected
    log_bias_field = bias_corrector.GetLogBiasFieldAsImage(raw_img_sitk)
    corrected_image_full_resolution = raw_img_sitk / sitk.Exp( log_bias_field )
    # bias field
    temp = sitk.Exp(log_bias_field)
    bias_field_image = sitk.Mask(temp, head_mask)
    return corrected_image_full_resolution, bias_field_image

In [5]:
def normalize(arr, N=255, eps=1e-6):
    """
    TO normalize an image by mapping its [Min,Max] into the interval [0,255]
    :param arr: Input (2D or 3D) array of image
    :param N: Scaling factor
    :param eps:
    :return: Normalized Image
    """
    # N=255
    # eps=1e-6
    arr = arr.astype(np.float32)
    output = N*(arr-np.min(arr))/(np.max(arr)-np.min(arr)+eps)
    return output

In [6]:
def process_image(input_image, output_path, is_pet):
    data = sitk.GetArrayFromImage(input_image)
    # shift the data up so that all intensity values turn positive
    data -= np.min(data)
    # Removing the outliers with a probability of occuring less than 5e-3 through histogram computation
    histo, bins = np.histogram(data.flatten(), 10)
    histo = normalize(histo)
    Bin = bins[np.min(np.where(histo < 5e-3))]
    data = np.clip(data, 0, Bin)
    image = sitk.GetImageFromArray(data)
    image.SetSpacing(input_image.GetSpacing())
    image.SetOrigin(input_image.GetOrigin())
    image.SetDirection(input_image.GetDirection())

    if is_pet:
        # If it's a PET image, just write it to the output path
        sitk.WriteImage(image, output_path)
    else:
        # If it's not a PET image, apply bias field correction

        # UNCOMMENT NEXT TWO LINES TO APPLY BIAS FIELD CORRECTION
        # corrected_image, _ = correct_bias_field(image)
        # sitk.WriteImage(corrected_image, output_path)

        sitk.WriteImage(image, output_path)

### Iterate over all patients mri and correct them

In [7]:
corrected_dataset_path = '/local_ssd/practical_wise24/prostate_cancer/mha_corrected_images/'
# corrected_dataset_path = './mha_corrected_images/'
root_dir = '../../../data/ProstateData/BREST patients/'
mha_images_path = '/local_ssd/practical_wise24/prostate_cancer/mha_raw_images'
# mha_images_path = './mha_raw_images'

In [8]:
with ProcessPoolExecutor(max_workers=os.cpu_count()) as executor:
    for patient_folder in os.listdir(root_dir):
        files_path = os.path.join(mha_images_path, patient_folder)
        write_file_path = os.path.join(corrected_dataset_path, patient_folder)

        if not os.path.exists(write_file_path):
            os.makedirs(write_file_path)
        else:
            continue # we assume that if the folder exists it has already the corrected files

        tasks = []

        for mha_file_name in os.listdir(files_path):
            image = sitk.ReadImage(os.path.join(files_path, mha_file_name), sitk.sitkFloat32)
            output_path = os.path.join(write_file_path, mha_file_name)
            is_pet = 'PET' in mha_file_name
            task = executor.submit(process_image, image, output_path, is_pet)
            tasks.append(task)

        # Wait for all tasks to complete
        for task in tasks:
            task.result()

#### Check all files are created in the corrected dataset

In [11]:
raw_dir = '/local_ssd/practical_wise24/prostate_cancer/mha_raw_images/'
corrected_dir = '/local_ssd/practical_wise24/prostate_cancer/mha_corrected_images/'
# raw_dir = './mha_raw_images/'
# corrected_dir = './mha_corrected_images/'
for patient_folder in os.listdir(raw_dir):
    raw_files_path = os.path.join(raw_dir, patient_folder)
    corrected_files_path = os.path.join(corrected_dir, patient_folder)

    if not os.path.exists(corrected_files_path):
        print('Directory does not exist: ' + corrected_files_path)
    else:
        if len(os.listdir(raw_files_path)) != len(os.listdir(corrected_files_path)):
            print(os.listdir(raw_files_path))
            print('-'* 30)
            print(os.listdir(corrected_files_path))
            print('*'*30)